# 패키지 설치하기

In [ ]:
!pip install ratsnlp

# 환경 설정

모델 hyperparameter와 저장 위치 등 설정 정보를 선언

In [2]:
from ratsnlp.nlpbook.ner import NERDeployArguments

In [5]:
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/content/drive/Othercomputers/내 컴퓨터/Chapter 6. Named Entity Recognition/checkpoint-ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /content/drive/Othercomputers/내 컴퓨터/Chapter 6. Named Entity Recognition/checkpoint-ner/epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: /content/drive/Othercomputers/내 컴퓨터/Chapter 6. Named Entity Recognition/checkpoint-ner/label_map.txt


# 모델 불러오기

파인튜닝을 마친 모델 불러오기

In [6]:
import torch
from transformers import BertConfig, BertForTokenClassification

In [7]:
# 체크포인트 로드
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

# BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForTokenClassification(pretrained_model_config)     # 모델 초기화
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})     # 체크포인트 주입
model.eval()        # 평가모드

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

토크나이저 준비

In [8]:
from transformers import BertTokenizer

In [9]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 레이블 맵 작성

범주 인덱스를 범주명과 매칭하는 사전을 만든다.

In [10]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}

for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
    
  id_to_label[idx] = label

In [15]:
id_to_label

{0: '[CLS]',
 1: '[SEP]',
 2: '[PAD]',
 3: '[MASK]',
 4: 'O',
 5: '인명',
 6: '기타 수량표현',
 7: '기타',
 8: '기관명',
 9: '날짜',
 10: '지명',
 11: '통화',
 12: '비율',
 13: '시간',
 14: '기간',
 15: '인명',
 16: '기타 수량표현',
 17: '기타',
 18: '기관명',
 19: '날짜',
 20: '지명',
 21: '통화',
 22: '비율',
 23: '시간',
 24: '기간'}

# 인퍼런스 함수

In [11]:
def inference_fn(sentence):
    
    # 입력 문장을 토큰화
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )

    # 평가 모드로 Gradient 업데이트 안함
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})  # 토큰화 된 입력 문장을 텐서로 변환 후 모델에 입력
        probs = outputs.logits[0].softmax(dim=1)

        top_probs, preds = torch.topk(probs, dim=1, k=1)    # 가장 높은 확률 값과 인덱스 구하기
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])    # 토큰 인덱스 시퀀스(int형 리스트) -> 토큰 시퀀스 변환(str형 리스트)
        predicted_tags = [id_to_label[pred.item()] for pred in preds]       # 개체명 인덱스 시퀀스(int형 리스트) -> 개체명 시퀀스 변환(str형 리스트)
        result = []

        # 특수 토큰을 제외한 토큰 닫닫에 대해 모델이 예측한 개체명과 확률값 반환
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
                
    return {
        "sentence": sentence,
        "result": result,
    }

# 웹서비스 개시

`ngrok` 은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구이다. 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 입력해야 한다.

In [12]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

인퍼런스 함수를 Flask를 통해 웹서비스로 만든다.

In [13]:
from ratsnlp.nlpbook.ner import get_web_service_app

In [14]:
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.ner.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f975-34-78-145-104.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Apr/2022 13:06:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:06:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:07:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:07:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Apr/2022 13:07:37] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:11:54] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:13:09] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:13:42] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:14:45] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 13:15:29] "POST /api HTTP/1.1" 200 -
